# "Настройка" BEIR

In [ ]:
""" # for beir run
!pip install datasets
!pip install beir
!pip install gdown """

In [1]:
!git clone https://github.com/kngrg/rus_beir.git
import sys 
sys.path.append("/kaggle/working/rus_beir")
# for local loaded lib 
!pip install -r /kaggle/working/rus_beir/requirements.txt

Cloning into 'rus_beir'...
remote: Enumerating objects: 2697, done.
remote: Counting objects: 100% (663/663), done.
remote: Compressing objects: 100% (142/142), done.
remote: Total 2697 (delta 576), reused 547 (delta 516), pack-reused 2034
Receiving objects: 100% (2697/2697), 39.82 MiB | 14.90 MiB/s, done.
Resolving deltas: 100% (1707/1707), done.
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.2/480.2 kB 8.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.5/64.5 kB 3.3 MB/s eta 0:00:00
  Created wheel for pytrec_eval: filename=pytrec_eval-0.5-cp310-cp310-linux_x86_64.whl size=293135 sha256=8745f5077c9c5a30780a2e33d024d716e80e9e4ec5ff966bb744b9cc341a1be5
  Stored in directory: /root/.cache/pip/wheels/51/3a/cd/dcc1ddfc763987d5cb237165d8ac249aa98a23ab90f67317a8
Successfully built pytrec_eval


In [10]:
""" import os
import gdown
import zipfile

#full corpus
#url = "https://drive.google.com/uc?export=download&id=1RkR7YQ2AewNg96MmHF7sksCC_oxxob5V"

#corpus_100
url = 'https://drive.google.com/uc?export=download&id=1l-dVU6GSKz5REXbOZ3JR4MEKV0TO0VKJ'

out_dir = os.path.join(os.getcwd(), "datasets")
os.makedirs(out_dir, exist_ok=True)
#output_file_path = os.path.join(out_dir, "rus-mmarco.zip")
output_file_path = os.path.join(out_dir, "rus-mmarco-mini.zip")

gdown.download(url, output_file_path, quiet=False)

#zip_path = os.path.join(out_dir, "rus-mmarco.zip")
zip_path = os.path.join(out_dir, "rus-mmarco-mini.zip")

print(zip_path)
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(out_dir)
print(f"Dataset extracted here: {out_dir}") """

SyntaxError: invalid syntax (4070642540.py, line 1)

In [2]:
from tqdm import tqdm
def CorpusToDict(corpus):
    corpus_dict = {}
    for row in tqdm(corpus, desc="corpus->dict"):
        corpus_dict[str(row['id'])] = {'text': row['text'], 'title': row['title'], 'processed_text': row['processed_text']}
    return corpus_dict

def QueriesToDict(queries):
    queries_dict = {}
    for row in tqdm(queries, desc="queries->dict"):
        queries_dict[str(row['id'])] = row['text']
    return queries_dict

In [3]:
# Load dataset via HF
from beir.datasets.data_loader_hf import HFDataLoader

corpus, queries, qrels = HFDataLoader(hf_repo="kngrg/rus-mmarco-mini", hf_repo_qrels="kngrg/rus-mmarco-mini-qrels", streaming=False,
                                                  keep_in_memory=False).load(split='train')
corpus = CorpusToDict(corpus)
queries = QueriesToDict(queries)

Generating train split:   0%|          | 0/100 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/100 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/100 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/100 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/100 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Filter:   0%|          | 0/100 [00:00<?, ? examples/s]

queries->dict: 100%|██████████| 100/100 [00:00<00:00, 9070.73it/s]


In [11]:
""" from beir.datasets.data_loader import GenericDataLoader

data_path = "datasets/rus-mmarco-mini"
corpus1, queries1, qrels1 = GenericDataLoader(data_path).load(split="train") # or split = "train"
"""

SyntaxError: incomplete input (557948234.py, line 1)

# ElasticSearch

In [4]:
%%bash

wget -q https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-oss-7.9.2-linux-x86_64.tar.gz
wget -q https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-oss-7.9.2-linux-x86_64.tar.gz.sha512
tar -xzf elasticsearch-oss-7.9.2-linux-x86_64.tar.gz
sudo chown -R daemon:daemon elasticsearch-7.9.2/
shasum -a 512 -c elasticsearch-oss-7.9.2-linux-x86_64.tar.gz.sha512

elasticsearch-oss-7.9.2-linux-x86_64.tar.gz: OK


In [5]:
%%bash --bg

sudo -H -u daemon elasticsearch-7.9.2/bin/elasticsearch

In [6]:
import time

# Sleep for few seconds to let the instance start.
time.sleep(20)

In [7]:
%%bash

ps -ef | grep elasticsearch

root         362     360  0 17:40 ?        00:00:00 sudo -H -u daemon elasticsearch-7.9.2/bin/elasticsearch
daemon       363     362 99 17:40 ?        00:00:27 /kaggle/working/elasticsearch-7.9.2/jdk/bin/java -Xshare:auto -Des.networkaddress.cache.ttl=60 -Des.networkaddress.cache.negative.ttl=10 -XX:+AlwaysPreTouch -Xss1m -Djava.awt.headless=true -Dfile.encoding=UTF-8 -Djna.nosys=true -XX:-OmitStackTraceInFastThrow -XX:+ShowCodeDetailsInExceptionMessages -Dio.netty.noUnsafe=true -Dio.netty.noKeySetOptimization=true -Dio.netty.recycler.maxCapacityPerThread=0 -Dio.netty.allocator.numDirectArenas=0 -Dlog4j.shutdownHookEnabled=false -Dlog4j2.disable.jmx=true -Djava.locale.providers=SPI,COMPAT -Xms1g -Xmx1g -XX:+UseG1GC -XX:G1ReservePercent=25 -XX:InitiatingHeapOccupancyPercent=30 -Djava.io.tmpdir=/tmp/elasticsearch-15970793861848470660 -XX:+HeapDumpOnOutOfMemoryError -XX:HeapDumpPath=data -XX:ErrorFile=logs/hs_err_pid%p.log -Xlog:gc*,gc+age=trace,safepoint:file=logs/gc.log:utctime,pid,tags

In [8]:
%%bash

curl -sX GET "localhost:9200/"

{
  "name" : "6f400d211fe1",
  "cluster_name" : "elasticsearch",
  "cluster_uuid" : "U0OlV6vwQW6feZU5SYiQnQ",
  "version" : {
    "number" : "7.9.2",
    "build_flavor" : "oss",
    "build_type" : "tar",
    "build_hash" : "d34da0ea4a966c4e49417f2da2f244e3e97b4e6e",
    "build_date" : "2020-09-23T00:45:33.626720Z",
    "build_snapshot" : false,
    "lucene_version" : "8.6.2",
    "minimum_wire_compatibility_version" : "6.8.0",
    "minimum_index_compatibility_version" : "6.0.0-beta1"
  },
  "tagline" : "You Know, for Search"
}


In [9]:
from beir.retrieval.search.lexical import BM25Search as BM25
from beir.retrieval.evaluation import EvaluateRetrieval

#### Provide parameters for elastic-search
hostname = "localhost"
index_name = "mmarco"
initialize = True # True, will delete existing index with same name and reindex all documents

model = BM25(index_name=index_name, hostname=hostname, initialize=initialize, language="russian")
retriever = EvaluateRetrieval(model)

#### Retrieve dense results (format of results is identical to qrels)
results = retriever.retrieve(corpus, queries)

ValueError: URL must include a 'scheme', 'host', and 'port' component (ie 'https://localhost:9200')

In [ ]:
#### Evaluate your retrieval using NDCG@k, MAP@K ...
ndcg, _map, recall, precision = retriever.evaluate(qrels, results, retriever.k_values)

In [ ]:
metrics = {"ndcg": ndcg, "_map": _map, "recall": recall, "precision": precision}
for metric in metrics.keys():
    for it_num, it_val in zip(metrics[metric], metrics[metric].values()):
        print(it_num, it_val )
    print('\n')
